# David Burton's Zombie Invasion

# Theory Overview

This is where I describe the math and how it will all work

# Program Implementation

These are the functions that I am importing to use in solvign the problems.

In [7]:
from pylab import array,sqrt
from random import randint

**These are my functions for modeling the population dynamics of a zombie invasion.**

**eurich** is my Euler-Richardson function that is used to integrate a function (f) that contains a variable (x) and a current time (t) forward in time by a specific change in time (dt). Eurich returns an output of the new variable x at a time that is some change in time (dt) forward from the current time (t).

In [84]:
#euler richardson function
def eurich(f,x,t,dt):
    #x["x"] is teh value of that thing, all else in the dictionary are modifiers and stuff
    tm=t+(dt/2) #calculatign the midpoint time
    #fudge factor for two species interactions
    if len(f(x,t))>1:
        #somehow, make two species interactions happen
        #perhaps store an array for each population as (pop, popchange)
        xm_y=x["Y"]+f(x,t)[1]*(dt/2)
        xm_x=x["Y"]+f(x,t)[0]*(dt/2) #calculatign the midpoint position
        x["Y"]=x["Y"]+fm[1]*dt #by convention, the predator is always Y
        x["X"]=x["X"]+fm[0]*dt #by convention, the prey is always X
    else: #we always act on the predator species by default
        xm=x["Y"]+f(x,t)*(dt/2) #calculatign the midpoint position
        fm=f(xm,tm) #calculatign the midpoint slopes
        x["Y"]=x["Y"]+fm*dt #calculatign the final xposition as estimated by the slope at the midpoint

#to make it so that I don't need another version of eurich for every function below, we're going
#to make x a dictionary that can get unpacked for every function

**rate-susp** is my rate of suspetibility function, which is based off of the following population dynamic equation:$$\frac{dS}{dt}= -\frac{\beta IS}{N}$$ It takes in beta (the rate of infection), I (the infected population), S (the susceptible population), and N (the total population) as inputs, and it returns as an output the rate of change in the susceptible population.

In [66]:
def rate_susp(x,t):
    beta,I,S,N=x["beta"],x["I"],x["S"],x["N"]
    return -(beta*I*S)/N

For example, if the rate of infection (beta) is 0.03, the infected population (I) is 100, the susceptible population is 25, and the total population is 150, then the rate of change in the susceptible population will be:

In [67]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150}
time=0 #starting time is zero
print(rate_susp(x,time)) #printing the output

-0.5


**rate_infected** is my rate of infection function, which is based off of the following population dynamic equation: $$\frac{dI}{dt}= \frac{\beta IS}{N} -\gamma I$$ It takes in beta (the rate of infection), I (the infected population), S (the susceptible population), N (the total population), gamma (the rate at which people recover) as inputs, and it returns as an output the rate at which people become infected.

In [68]:
def rate_infected(x,t):
    beta,I,S,N,gamma=x["beta"],x["I"],x["S"],x["N"],x["gamma"]
    return ((beta*I*S)/N)-(gamma*I)

For example, if the rate of infection (beta) is 0.03, the infected population (I) is 100, the susceptible population is 25, the total population is 150, and the recovery rate (gamma) is 0.001, then the rate of change in the infected population will be:

In [69]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150,"gamma":0.001}
time=0
print(rate_infected(x,time)) #printing the output

0.4


**rate_recovery** is my rate of recovery function, which is based off of the following population dynamic equation:
 $$\frac{dR}{dt}= \gamma I$$ It takes in I (the infected population) and gamma (the rate at which people recover) as inputs, and it returns as an output the rate at which people recover.

In [70]:
def rate_recovery(x,t):
    I,gamma=x["I"],x["gamma"]
    return gamma*I

For example, if infected population (I) is 100 and the recovery rate (gamma) is 0.001, then the rate at which people recover will be:

In [71]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150,"gamma":0.001}
time=0
print(rate_recovery(x,time)) #printing the output

0.1


**pop_change_1sp** is my rate of population change for one species function without a carrying capacity, which is based off of the following population dynamic equation: $$\frac{dx}{dt}= rx$$ It takes in r (the birth rate minus the death rate) and x (the population), then it returns as an output the rate at which the population changes.

In [72]:
def pop_change_1sp(x,t):
    r,X=x["r"],x["X"]
    return r*X

For example, if the birth rate is 0.5, the death rate is 0.15 and the population (pop) is 125, then the population change will be:

In [73]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150,"gamma":0.001,"r":0.35,"X":125}
time=0
print(pop_change_1sp(x,time)) #printing the output

43.75


**pop_change_1spK** is my rate of population change for one species function with a carrying capacity, which is based off of the following population dynamic equation: $$\frac{dx}{dt}= rx(1-\frac{x}{K})$$ It takes in r (the birth rate minus the death rate), x (the population), and K (the carrying capacity), and it returns as an output the rate at which the population changes.

In [74]:
def pop_change_1spK(x,t):
    r,X,K=x["r"],x["X"],x["K"]
    return r*X*(1-(X/K))

For example, if the birth rate is 0.5, the death rate is 0.15, the population (pop) is 125, and the carrying capacity is 150, then the population change will be:

In [76]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150,"gamma":0.001,"r":0.35,"X":125,"K":150}
time=0
print(pop_change_1spK(x,time)) #printing the output

7.291666666666665


**pop_change_2sp** is my predator-prey population interactions function, which is based off of the population dynamic equations: $$\frac{dx}{dt}= \alpha x-\beta xy$$ $$\frac{dy}{dt}= \delta xy-\gamma y $$It takes alpha (the birth rate minus the death rate for the prey species-x), beta (the success rate of how often the predator species-y gets its prey species-x), delta (the rate that predator can convetr prey into more predator), gamma (the death rate of teh predator species-y), x (the population of the prey species), and y (the population of the predator species)as inputs. It returns as an output the rate at which population x and population y change.

In [77]:
def pop_change_2sp(x,t):
    alpha1,beta1,delta1,gamma1,X,Y=x["alpha1"],x["beta1"],x["delta1"],x["gamma1"],x["X"],x["Y"]
    change_x=(alpha1*X)-(beta1*X*Y)
    change_y=(delta1*X*Y)-(gamma1*Y)
    return [change_x,change_y]

For example, if birth minus the death rate of the prey species (alpha1) is 0.5, the success rate of predator species catchign prey species (beta1) is 0.01, the rate of predator species converting prey into more predators (delta1) is 0.001, the death rate of the predator species (gamma1) is 0.1, the population of prey species (X) is 125, and the population of the predator species (Y) is 25, then the rate of change of each population will be:

In [83]:
#Our Inputs
x={"beta":0.03,"I":100,"S":25,"N":150,"gamma":0.001,"r":0.35,"X":125,"K":150,"alpha1":0.5,"beta1":0.01,"delta1":0.001,"gamma1":0.1,"Y":25}
time=0
print(pop_change_2sp(x,time)) #printing the output

[31.25, 0.625]


**zombie_attack** is the zombie attack function which models what occurs after a change in time (deltaT) from the starting time (time). We use the following assumptions in using our function: <br>
-all humans are susceptible <br>

In [36]:
#Our inputs
#increment of tiem change (dt)
#amoutn fo tiem to progerss forward (ft)
#current time (t)
#human population (x)
#fed zombie population (y)
#hungry zombie population (z)
#death rate of hungry zombies]
#conversion of fed zombies to hungry zombies

def zombie_attack(x,y,z,t,ft,dt):
    #unpacking the populations
    X=x #human population
    Y=y["Y"] #fed zombie population
    Z=z["Y"] #hungry zombie population
    
    #setting human population to equal in both groups
    y["X"],z["X"]=X,X
    
    #moving forward
    while current_time<=t+ft: 
        #we integrate forward as long at the current time is less than the start t+ft
        #fed zombies scratch humans
        #we assume that the conversion rate of prey to pedator is one to one
        #we assume teh death rate of scratched zombies is their conversion to hungry zombeis
        #this is a two species interaction
        eurich()
    
        #set the human population to match in the other data set
    
        #hungry zombies attackign humans
        #we assume that the conversion 
    
        #fed zombies convertign to hungry zombies
        
        #moving forward in time
        
    return [X,Y,Z]
    

# Answering The Questions